In [5]:
def connect_to_db():

    import sqlalchemy
    import sqlite3
    
    # Create SQL Alchemy database connection
    db_url = f"sqlite:///C:/Users/JanseDanny/OneDrive/Documenten Danny/Football Fortunes/database/DEV/footballfortunes.sqlite"
    ##db_url = f"sqlite:////app/data/database/DEV/footballfortunes.sqlite"
    engine = sqlalchemy.create_engine(db_url, echo=True)
    sqal_connection = engine.connect()

    # Create Psychop connection
    psy_connection = sqlite3.connect("C:/Users/JanseDanny/OneDrive/Documenten Danny/Football Fortunes/database/DEV/footballfortunes.sqlite")
    ##psy_connection = sqlite3.connect("/app/data/database/DEV/footballfortunes.sqlite")
    
    return psy_connection, sqal_connection

def close_connection_db(psy_connection, sqal_connection):
    
    # Commit de transactie en sluit de verbinding
    sqal_connection.close()
    psy_connection.close()  

In [20]:
def quality_control(sqal_connection, controle_map):

    import pandas as pd
    import glob
    import os

    print(f'{controle_map} quality control gestart')
    
    current_directory = os.getcwd()
    parent_directory = os.path.dirname(os.getcwd())
    path = f'{parent_directory}/04 QualityDSA/{controle_map}'
    sql_files = glob.glob(path + '\*.sql')
    print(path)
    print(sql_files)
    aantal_issues = 0

    for sql_file in sql_files:
        
    # Lees het SQL-bestand in en voer de queries uit
        with open(sql_file) as file:
            sql_script = file.read()
            resultaten = pd.read_sql(sql_script, sqal_connection)
                    
        for i, resultaat in resultaten.iterrows():
            if resultaat[0] == 0:
                print(f'   TOP! {resultaat[1]}')
            else: 
                print(f'   OEPS! {resultaat[1]}') 
                aantal_issues += 1
       
    print(f'{controle_map} quality control gereed')
    
    return(aantal_issues)

In [21]:
# connectie maken met db
psy_connection = connect_to_db()[0]
sqal_connection = connect_to_db()[1]


# kwaliteitcontrole dsa
quality_control(sqal_connection, 'sql')


close_connection_db(psy_connection, sqal_connection)

sql quality control gestart
c:\Users\JanseDanny\OneDrive\Documenten Danny\Lead Data Engineer\footballfortunes/04 QualityDSA/sql
['c:\\Users\\JanseDanny\\OneDrive\\Documenten Danny\\Lead Data Engineer\\footballfortunes/04 QualityDSA/sql\\01 dsa_fd_stats.sql', 'c:\\Users\\JanseDanny\\OneDrive\\Documenten Danny\\Lead Data Engineer\\footballfortunes/04 QualityDSA/sql\\01 dsa_odd_api.sql', 'c:\\Users\\JanseDanny\\OneDrive\\Documenten Danny\\Lead Data Engineer\\footballfortunes/04 QualityDSA/sql\\01 dwa_teams_wedkantoren.sql']
2024-10-19 16:10:00,645 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-19 16:10:00,645 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("-- maak een tabel met volgnummers voor analyse van dubbele waarden in fd_stats
-- en een lijst met aantallen per seizoen in fd_stats voor vergelijk met controletabel
-- en een lijst met teamnamen in fd_stats voor vergelijk met dwa_d_teams
with 
	dubbele_fd_stats as (
		select
		row_number() over (partition by hometeam, awa